In [77]:
from bson.objectid import ObjectId
from datetime import datetime
import random
import time

In [78]:
random.seed(3)
def prefix_crud_timestamp_suffix(key):
    prefix = key[:3]
    crud = key[3:4]
    #hyphens = [i for i in range(len(key[:4])) if key[:4].startswith('-'', i)]
    hyphen1 = key.find('-')
    hyphen2 = key[5:].find('-')
    timestamp = key[hyphen1+1:hyphen1+1+hyphen2]
    suffix = key[hyphen1+hyphen2+2:]
    return prefix, crud, timestamp, suffix #coll, op, time, guid

## seconds since midnight, simulate non-contiguous times
def ssm():
    now = datetime.now()
    midnight = now.replace(hour=0, minute=0, second=0, microsecond=0)
    return str((now - midnight).seconds + random.randint(0, 1000))

words = dict()
def enter_words(en, zh = None, hi = None):
    uid = ('zhon-' if zh != None else 'hind-' if hi != None else 'oops-') + ssm() + '-' + str(ObjectId())
    words[uid] = (
        dict(english = en, chinese = zh, _id = uid) if zh != None else 
        dict(english = en, hindi = hi, _id = uid) if hi != None else
        dict(_id = uid)
    )

In [79]:
# read chinese and hindi sentences
# please make sure cmn.txt and hin.txt are under data, data and Assignment3_HenanMu.ipynb are under the same directory

file1 = open('data/cmn.txt', 'r', encoding='utf8')
lines = file1.readlines()
file1.close()

for i,l in enumerate(lines):
    t2 = l.split('\t')
    enter_words(t2[0][:-1], zh = t2[1][:-1])

file1 = open('data/hin.txt', 'r', encoding='utf8')
lines = file1.readlines()
file1.close()

for i,l in enumerate(lines):
    t2 = l.split('\t')
    enter_words(t2[0][:-1], hi = t2[1][:-1])

separated = dict()
separated['chinese'] = {k:v for k,v in words.items() if k.startswith('zhon')}
separated['hindi'] = {k:v for k,v in words.items() if k.startswith('hind')}

In [80]:
# Question 1 
def get_sentences_v1(n, s_time):
    my_result = dict()
    chinese_dict = dict()
    hindi_dict = dict()
    count_of_chinese = 0;
    count_of_hindi = 0;
    for i, (k,v) in enumerate(separated['chinese'].items()):
        if count_of_chinese == n:
            break
        else:
            prefix, crud, timestamp, suffix = prefix_crud_timestamp_suffix(k)
            if (int(timestamp) > s_time):
                chinese_dict[k] = v
                count_of_chinese += 1
    for i, (k,v) in enumerate(separated['hindi'].items()):
        if count_of_hindi == n:
            break
        else:
            prefix, crud, timestamp, suffix = prefix_crud_timestamp_suffix(k)
            if (int(timestamp) > s_time):
                hindi_dict[k] = v
                count_of_hindi += 1
    my_result['chinese'] = chinese_dict
    my_result['hindi'] = hindi_dict
    return my_result

print(get_sentences_v1(3, 1000))

{'chinese': {'zhon-1470-632be28b5b16e50aedb019b7': {'english': 'Hi', 'chinese': '嗨', '_id': 'zhon-1470-632be28b5b16e50aedb019b7'}, 'zhon-1833-632be28b5b16e50aedb019b8': {'english': 'Hi', 'chinese': '你好', '_id': 'zhon-1833-632be28b5b16e50aedb019b8'}, 'zhon-1784-632be28b5b16e50aedb019b9': {'english': 'Run', 'chinese': '你用跑的', '_id': 'zhon-1784-632be28b5b16e50aedb019b9'}}, 'hindi': {'hind-1489-632be28b5b16e50aedb06ff3': {'english': 'Wow', 'hindi': 'वाह', '_id': 'hind-1489-632be28b5b16e50aedb06ff3'}, 'hind-1474-632be28b5b16e50aedb06ff4': {'english': 'Help', 'hindi': 'बचाओ', '_id': 'hind-1474-632be28b5b16e50aedb06ff4'}, 'hind-1833-632be28b5b16e50aedb06ff5': {'english': 'Jump', 'hindi': 'उछलो', '_id': 'hind-1833-632be28b5b16e50aedb06ff5'}}}


In [81]:
%%time
# Question 2
def get_sentences_v2_Dua_Lipa(n, s_time):
    my_result = {'chinese': dict(), 'hindi': dict()}
    prefix_to_key_of_result = {'zhon': 'chinese', 'hind': 'hindi'}
    while (k,v) in enumerate(words.items()) and not(len(my_result['chinese']) == n and len(my_result['hindi']) == n):
        prefix, crud, timestamp, suffix = prefix_crud_timestamp_suffix(k)
        my_result[prefix_to_key_of_result[prefix]][k] = (v if int(timestamp) > s_time else null)
    return my_result

print(get_sentences_v1(3, 1000))
time.sleep(1)

{'chinese': {'zhon-1470-632be28b5b16e50aedb019b7': {'english': 'Hi', 'chinese': '嗨', '_id': 'zhon-1470-632be28b5b16e50aedb019b7'}, 'zhon-1833-632be28b5b16e50aedb019b8': {'english': 'Hi', 'chinese': '你好', '_id': 'zhon-1833-632be28b5b16e50aedb019b8'}, 'zhon-1784-632be28b5b16e50aedb019b9': {'english': 'Run', 'chinese': '你用跑的', '_id': 'zhon-1784-632be28b5b16e50aedb019b9'}}, 'hindi': {'hind-1489-632be28b5b16e50aedb06ff3': {'english': 'Wow', 'hindi': 'वाह', '_id': 'hind-1489-632be28b5b16e50aedb06ff3'}, 'hind-1474-632be28b5b16e50aedb06ff4': {'english': 'Help', 'hindi': 'बचाओ', '_id': 'hind-1474-632be28b5b16e50aedb06ff4'}, 'hind-1833-632be28b5b16e50aedb06ff5': {'english': 'Jump', 'hindi': 'उछलो', '_id': 'hind-1833-632be28b5b16e50aedb06ff5'}}}
CPU times: user 2.37 ms, sys: 1.59 ms, total: 3.96 ms
Wall time: 1 s
